# Stock Performance

See the gains/losses of your stocks in certain period (e.g. last 1 month, 3 months, 1 year etc.) or a custom date range

#### Enter the list of stock tickers you would like to analyze

In [8]:
crazy_stocks = ['AAPL', 'AMZN', 'BABA', 'BIGC', 'DOCU', 'ETSY', 'FB', 'GOOG',
                'JD', 'LYFT', 'MSFT', 'NFLX', 'NIO', 'NVDA', 'PINS', 'PYPL', 
                'ROKU', 'SHOP', 'SPLK', 'SQ', 'TSLA', 'UBER', 'W', 'WMT', 'ZM', 
                'VGT', 'QQQ', 'TQQQ', 'FNGU']

In [9]:
""" Download the stock prices using yfinance API """

import numpy as np
import pandas as pd

from datetime import date
from datetime import timedelta

import yfinance as yf


tickers = crazy_stocks

today = '{}'.format(date.today())
y1_ago = '{}'.format(date.today() - timedelta(days = 365))

tickers_str = ' '.join(tickers)

data = yf.download(tickers_str, y1_ago, today, group_by='ticker')


print("Closing Prices")
c_prices = data.iloc[-1].loc[(slice(None), 'Close')].sort_index()
print(c_prices.round(2))

[*********************100%***********************]  29 of 29 completed
Closing Prices
AAPL     120.96
AMZN    3294.62
BABA     281.39
BIGC     106.98
DOCU     216.26
ETSY     112.04
FB       282.73
FNGU     194.50
GOOG    1591.04
JD        80.00
LYFT      28.97
MSFT     214.25
NFLX     516.05
NIO       17.98
NVDA     504.90
PINS      34.38
PYPL     191.84
QQQ      283.58
ROKU     159.91
SHOP     973.02
SPLK     200.14
SQ       146.39
TQQQ     141.64
TSLA     418.32
UBER      33.24
VGT      313.29
W        260.94
WMT      142.83
ZM       369.89
Name: 2020-09-04 00:00:00, dtype: float64


In [76]:
""" Compute last 1m, 3m, 6m and 1year gains """

y1_ago_price = data.iloc[0].loc[(slice(None), 'Close')]
m6_ago_price = data.iloc[-26 * 5].loc[(slice(None), 'Close')]
m3_ago_price = data.iloc[-13 * 5].loc[(slice(None), 'Close')]
m1_ago_price = data.iloc[-4 * 5].loc[(slice(None), 'Close')]
curr_price = data.iloc[-1].loc[(slice(None), 'Close')]


m1_gain = (curr_price - m1_ago_price) / m1_ago_price
m3_gain = (curr_price - m3_ago_price) / m3_ago_price
m6_gain = (curr_price - m6_ago_price) / m6_ago_price
y1_gain = (curr_price - y1_ago_price) / y1_ago_price

curr_date = np.datetime_as_string(data.index.values[-1], unit='D')

df = pd.DataFrame([curr_price, m1_gain, m3_gain, m6_gain, y1_gain],
                  index=[curr_date,  '1m', '3m', '6m', '1y'])

df = df.T.sort_values(by='1m', ascending=False)

format_dict = { col_name : '{:.2%}' for col_name in ['1m', '3m', '6m', '1y', 'Gain']}
df.style.format(format_dict)

,2020-09-04,1m,3m,6m,1y
ZM,369.89,47.96%,78.17%,216.69%,370.30%
TSLA,418.32,47.44%,136.16%,179.07%,802.37%
BIGC,106.98,39.44%,nan%,nan%,nan%
FNGU,194.5,37.26%,127.49%,141.40%,420.61%
JD,80,30.10%,35.50%,82.19%,154.13%
NIO,17.98,26.53%,221.65%,364.60%,472.61%
BABA,281.39,13.40%,28.17%,32.76%,58.28%
TQQQ,141.64,13.06%,57.57%,59.34%,121.35%
NVDA,504.9,13.05%,41.51%,77.46%,179.72%
DOCU,216.26,9.01%,54.87%,141.50%,269.23%


In [84]:
""" Sort the stocks by 1 year gains"""

df = df.sort_values(by='1y', ascending=False)
df.style.format(format_dict)

,2020-09-04,1m,3m,6m,1y
TSLA,418.32,47.44%,136.16%,179.07%,802.37%
NIO,17.98,26.53%,221.65%,364.60%,472.61%
FNGU,194.5,37.26%,127.49%,141.40%,420.61%
ZM,369.89,47.96%,78.17%,216.69%,370.30%
DOCU,216.26,9.01%,54.87%,141.50%,269.23%
NVDA,504.9,13.05%,41.51%,77.46%,179.72%
SHOP,973.02,-3.24%,33.70%,89.96%,171.29%
JD,80,30.10%,35.50%,82.19%,154.13%
SQ,146.39,5.06%,62.78%,84.70%,138.15%
AAPL,120.96,7.30%,45.95%,59.82%,125.91%


### This section lets you see the gains in the custom date range
Update the date1 and date2. Note both have to be withing 1 year as the data is fecteched just for a year

In [77]:
date1 = '2020-09-02'
date2 = '2020-09-04'

In [78]:
price1 = data.loc[date1].loc[(slice(None), 'Close')]
price2 = data.loc[date2].loc[(slice(None), 'Close')]
price1.name = date1
price2.name = date2

diff = (price2 - price1) / price1
diff.name = "Gain"


change = pd.concat([price1, price2, diff], axis=1)
change = change.sort_values(by="Gain", ascending=False)


change.style.format(format_dict)

,2020-09-02,2020-09-04,Gain
LYFT,28.61,28.97,1.26%
WMT,147.68,142.83,-3.28%
UBER,34.37,33.24,-3.29%
JD,83.21,80,-3.86%
BIGC,112,106.98,-4.48%
BABA,296.07,281.39,-4.96%
QQQ,302.76,283.58,-6.34%
TSLA,447.37,418.32,-6.49%
FB,302.5,282.73,-6.54%
NFLX,552.84,516.05,-6.65%
